In [ ]:
%pip install rpy2
%load_ext rpy2.ipython
# import subprocess
# subprocess.run('conda install -c conda-forge r-base', shell=True)
# !pip install rpy2
import numpy as np
import statsmodels.formula.api as smf
import rpy2
from rpy2.robjects import r, pandas2ri
pandas2ri.activate()

In [ ]:
%%R
install.packages("dagitty", quiet=TRUE)
library(dagitty, quiet=TRUE)

# Collider Bias

In [ ]:
%%R
g = dagitty("dag{ T -> C <- B }")
plot(g)

In [ ]:
#collider bias
num_samples = 1000000
talent = np.random.normal(size=num_samples)
beauty = np.random.normal(size=num_samples)
congeniality = talent + beauty + np.random.normal(size=num_samples) #congeniality
cond_talent = talent[congeniality > 0]
cond_beauty = beauty[congeniality > 0]
data = {"talent": talent, "beauty": beauty, "congeniality": congeniality, "cond_talent": cond_talent, "cond_beauty": cond_beauty}

print(smf.ols("talent ~ beauty", data).fit().summary())
print(smf.ols("talent ~ beauty + congeniality", data).fit().summary())
print(smf.ols("cond_talent ~ cond_beauty", data).fit().summary())

We can also use package Dagitty to illustrate collider bias, also known as M-bias.

In [ ]:
%%R
print(adjustmentSets( g, "T", "B" ))

## empty set -- we should not condition on the additional
## variable C.
## Generate data where C = .5T + .5B

d <- simulateSEM( g, .5 )

In [ ]:
data = rpy2.robjects.r["d"]
print(smf.ols("T ~ B", data).fit().summary()) # includes 0
print(smf.ols("T ~ B + C", data).fit().summary()) # does not include 0